In [ ]:
%load_ext autoreload
%autoreload 2

import networkx as nx
import pandas as pd
import numpy as np
from krxns.net_construction import construct_reaction_network, extract_compounds
from krxns.config import filepaths
from krxns.utils import str2int
from krxns.cheminfo import expand_unpaired_cofactors, draw_reaction, draw_molecule, standardize_smiles
from krxns.networks import SuperMultiDiGraph
import json
from IPython.display import SVG

In [ ]:
# Load unpaired cofs
unpaired_fp = filepaths['cofactors'] / "unpaired_cofactors_reference.tsv"
name_blacklist = [
    'acetyl-CoA',
]

unpaired_ref = pd.read_csv(
    filepath_or_buffer=unpaired_fp,
    sep='\t'
)

filtered_unpaired = unpaired_ref.loc[~unpaired_ref['Name'].isin(name_blacklist), :]
cofactors = expand_unpaired_cofactors(filtered_unpaired, k=10)

manual = {
    'N#N': 'N2',
    '[H][H]': 'H2',
    'S': 'hydrogen sufide',
    '[Cl-]': 'chloride',
    '[Na+]': 'sodium'
}

cofactors = {**cofactors, ** manual}

# Load cc sim mats
cc_sim_mats = {
    'mcs': np.load(filepaths['sim_mats'] / "mcs.npy"),
    'tanimoto': np.load(filepaths['sim_mats'] / "tanimoto.npy")
}

# Load known reaction data
with open(filepaths['data'] / 'sprhea_240310_v3_mapped.json', 'r') as f:
    krs = json.load(f)

# Load op connected reactions
with open(filepaths['connected_reactions'] / 'sprhea_240310_v3_mapped_operator.json', 'r') as f:
    op_cxns = str2int(json.load(f))

kcs, smi2id = extract_compounds(krs)

In [51]:
G = SuperMultiDiGraph()
edges, nodes = construct_reaction_network(operator_connections=op_cxns, reactions=krs, cofactors=cofactors)
G.add_nodes_from(nodes)
edge_keys = G.add_edges_from(edges)

In [94]:
def show_path(from_i, to_j, topk = 3):
    path = nx.shortest_path(G, from_i, to_j)
    for i in range(len(path) - 1):
        print(f"STEP {i+1} ")
        for edge in G.get_edges_between(path[i], path[i+1])[:topk]:
            print([(k, v) for k, v in edge.items() if k != 'smarts'])
            display(SVG(draw_reaction(edge['smarts'])))

In [ ]:
i = G.get_nodes_by_prop('name', 'pyruvate')[0]
j  = G.get_nodes_by_prop('name', 'succinate')[0]
show_path(i, j)

In [ ]:
i = G.get_nodes_by_prop('name', 'pyruvate')[0]
j  = G.get_nodes_by_prop('name', 'acetone')[0]
show_path(i, j)

In [ ]:
i = G.get_nodes_by_prop('name', 'pyruvate')[0]
j  = G.get_nodes_by_prop('name', 'L-lysine')[0]
show_path(i, j)

In [ ]:
sorted([node['name'] for id, node in G.nodes(data=True)], key=lambda x : len(x))

In [ ]:
id = G.get_nodes_by_prop('name', 'chorismate')[0]
for i in nx.ego_graph(G, id, radius=2):
    print(G.nodes[i])

In [ ]:
id = G.get_nodes_by_prop('name', 'chorismate')[0]
for i in nx.ego_graph(G, id, radius=2):
    print(f"PATH TO {G.nodes[i]['name']}")
    show_path(id, i)